In [2]:
src_df = pd.read_csv("data/train-balanced-sarcasm-cleaned.csv")

In [3]:
! pip install -r requirements.txt

     |████████████████████████████████| 175 kB 24.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 4.0 MB 67.9 MB/s            
     |████████████████████████████████| 67 kB 792 kB/s             
     |████████████████████████████████| 6.6 MB 74.6 MB/s            
     |████████████████████████████████| 880 kB 64.3 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=170372 sha256=9c71ca74c2dc5a51e72a2a09c552dcfe455e83fb2ec729ce5e96625086b969b6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/b7/c0/67/93ab86d74ab81bb91fd8611c497dbb7a4c26e4d941caddf70c
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=909303 sha256=ab8f278fa4fe3bbb183ab693145a19a05f7814c15ef85c3070acd9c8e74375ca
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4c/64/31/e9900a234b23fb3e9dc565d6114a9d6ff84a72dbdd356502b4
Successfully built emoji sacremos

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

model = AutoModelForMaskedLM.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [5]:
src_df.head(1)

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."


In [6]:
src_df.head(1)['parent_comment']

0    Yeah, I get that argument. At this point, I'd ...
Name: parent_comment, dtype: object

In [7]:
import re
import pickle
import emoji
import numpy as np

with open('Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)

def convert_emoticon_and_emoji_to_word(text):
    for emot in Emoticon_Dict:
        text = re.sub(r'('+emot+')', " ".join(Emoticon_Dict[emot].replace(",","").replace(":","").replace("andry", "angry").split(" ")), text)
    return emoji.demojize(text).replace(":", "").replace("_"," ")

In [9]:
Emoticon_Dict

{':‑\\)': 'Happy face or smiley',
 ':\\)': 'Happy face or smiley',
 ':-\\]': 'Happy face or smiley',
 ':\\]': 'Happy face or smiley',
 ':-3': 'Happy face smiley',
 ':3': 'Happy face smiley',
 ':->': 'Happy face smiley',
 ':>': 'Happy face smiley',
 '8-\\)': 'Happy face smiley',
 ':o\\)': 'Happy face smiley',
 ':-\\}': 'Happy face smiley',
 ':\\}': 'Happy face smiley',
 ':-\\)': 'Smile',
 ':c\\)': 'Happy face smiley',
 ':\\^\\)': 'Happy face smiley',
 '=\\]': 'Happy face smiley',
 '=\\)': 'Happy face smiley',
 ':‑D': 'Laughing, big grin or laugh with glasses',
 ':D': 'Happy face',
 '8‑D': 'Laughing, big grin or laugh with glasses',
 '8D': 'Laughing, big grin or laugh with glasses',
 'X‑D': 'Laughing, big grin or laugh with glasses',
 'XD': 'Laughing, big grin or laugh with glasses',
 '=D': 'Laughing, big grin or laugh with glasses',
 '=3': 'Laughing, big grin or laugh with glasses',
 'B\\^D': 'Laughing, big grin or laugh with glasses',
 ':-\\)\\)': 'Really happy:-DBig grin',
 ':‑\\(': '

In [8]:
convert_emoticon_and_emoji_to_word("I recommend uninstalling the old drivers before you install the new ones as well :)")

'I recommend uninstalling the old drivers before you install the new ones as well Happy face or smiley'

In [10]:
convert_emoticon_and_emoji_to_word("game is on 🔥 🔥")

'game is on fire fire'

In [12]:
convert_emoticon_and_emoji_to_word( "🍖 TEXT 🍞🍖 OTHER TEXT")

'meat on bone TEXT breadmeat on bone OTHER TEXT'

In [102]:
#Region to clean and store the comments in s3
from tqdm import tqdm
tqdm.pandas()
src_df['cleaned_comment'] = src_df['comment'].replace(np.nan, " ").progress_apply(convert_emoticon_and_emoji_to_word)
src_df['parent_cleaned_comment'] = src_df['parent_comment'].replace(np.nan, " ").progress_apply(convert_emoticon_and_emoji_to_word)

clean_file_name = "train-balanced-sarcasm-cleaned.csv"
src_df.to_csv(clean_file_name)

s3 = boto3.resource('s3')
s3.meta.client.upload_file(clean_file_name, bucket, f'data/{clean_file_name}')

100%|██████████| 1010826/1010826 [08:35<00:00, 1959.11it/s]


In [103]:
src_df.loc[176, ['comment', 'cleaned_comment']]

comment                              I recommend uninstalling the old drivers before you install the new ones as well :)
cleaned_comment    I recommend uninstalling the old drivers before you install the new ones as well Happy face or smiley
Name: 176, dtype: object